**submit.py**

In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from scipy.linalg import khatri_rao

################################
# Non Editable Region Starting #
################################
def my_fit(X_train, y0_train, y1_train):
    ################################
    #  Non Editable Region Ending  #
    ################################

    # Map training data to a higher dimensional space
    X_tr = my_map(X_train)

    # Train SVM for the first response
    model_0 = LinearSVC()
    model_0.fit(X_transformed, y0_train)

    # Train SVM for the second response
    model_1 = LinearSVC()
    model_1.fit(X_transformed, y1_train)

    # Extract the weights and intercepts
    weights_0 = model_0.coef_.ravel()
    intercept_0 = model_0.intercept_[0]

    weights_1 = model_1.coef_.ravel()
    intercept_1 = model_1.intercept_[0]

    return weights_0, intercept_0, weights_1, intercept_1

################################
# Non Editable Region Starting #
################################
def my_map(X):
    ################################
    #  Non Editable Region Ending  #
    ################################
    num_features = X.shape[1]
    num_samples = X.shape[0]
    num_mapped_features = num_features ** 2

    # Initialize matrix for mapped features
    mapped_features = np.zeros((num_samples, num_mapped_features))

    # Compute the Khatri-Rao product for each sample
    for i in range(num_samples):
        row = X[i, :].reshape(1, -1)
        kr_product = khatri_rao(row.T, row.T)
        mapped_features[i, :] = kr_product.flatten()

    return mapped_features

def validate():
    # Example file paths
    train_file = "public_trn.txt"
    test_file = "public_tst.txt"

    # Load data
    X_train, y0_train, y1_train = load_data(train_file)

    # Fit the model
    weight_0, intercept_0, weight_1, intercept_1 = my_fit(X_train, y0_train, y1_train)

    # Print the results
    print("Weights and bias for Response0:")
    print(weight_0, intercept_0)
    print("Weights and bias for Response1:")
    print(weight_1, intercept_1)

    # Load test data (assuming we want to validate on the same data format)
    X_test, y0_test, y1_test = load_data(test_file)

    # Map the test data
    X_test_mapped = my_map(X_test)

    # Predict responses
    y0_pred = np.sign(X_test_mapped @ weight_0 + intercept_0)
    y1_pred = np.sign(X_test_mapped @ weight_1 + intercept_1)

    # Calculate accuracy
    accuracy0 = np.mean((y0_pred > 0) == y0_test)
    accuracy1 = np.mean((y1_pred > 0) == y1_test)

    print("Test accuracy for Response0:", accuracy0)
    print("Test accuracy for Response1:", accuracy1)

def load_data(file_path):
    # Load training data
    data = np.loadtxt(file_path, delimiter=" ")
    X = data[:, :32]
    y0 = data[:, 32]
    y1 = data[:, 33]
    return X, y0, y1

if __name__ == "__main__":
    validate()

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Weights and bias for Response0:
[  1.05431279   0.04321148  -0.21883783 ...  -0.23336462   4.16179397
 -11.58055653] -0.4314787555438455
Weights and bias for Response1:
[ 0.33123174  0.56399271 -0.18142129 ...  0.24687877  0.8269801
 -1.77358191] -4.617529905349847
Test accuracy for Response0: 0.9855
Test accuracy for Response1: 0.9942


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


**Question 6**

In [ ]:
import numpy as np
import time
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load data function
def load_data(file_path):
    data = np.loadtxt(file_path, delimiter=" ")
    X = data[:, :32]
    y0 = data[:, 32]
    y1 = data[:, 33]
    return X, y0, y1




    ################################
def my_map(X):
    ################################
    #  Non Editable Region Ending  #
    ################################
    num_features = X.shape[1]
    num_samples = X.shape[0]
    num_mapped_features = num_features ** 2

    # Initialize the mapped features matrix
    mapped_features = np.zeros((num_samples, num_mapped_features))

    # Iterate over rows and compute Khatri-Rao product for each row
    for i in range(num_samples):
        row = X[i, :].reshape(1, -1)  # Reshape row to a 2D array
        row_result = khatri_rao(row.T, row.T)  # Compute Khatri-Rao for the row, transposed
        mapped_features[i, :] = row_result.flatten()

    return mapped_features

# Training and evaluation function
def train_and_evaluate(X_train, y0_train, y1_train, X_test, y0_test, y1_test, model, **kwargs):
    X_train_mapped = my_map(X_train)
    X_test_mapped = my_map(X_test)

    clf0 = model(**kwargs)
    clf1 = model(**kwargs)

    start_time = time.time()
    clf0.fit(X_train_mapped, y0_train)
    clf1.fit(X_train_mapped, y1_train)
    training_time = time.time() - start_time

    y0_pred = clf0.predict(X_test_mapped)
    y1_pred = clf1.predict(X_test_mapped)

    accuracy0 = accuracy_score(y0_test, y0_pred)
    accuracy1 = accuracy_score(y1_test, y1_pred)

    return training_time, accuracy0, accuracy1

# Load data
X_train, y0_train, y1_train = load_data("public_trn.txt")
X_test, y0_test, y1_test = load_data("public_tst.txt")

# Experiments setup
experiments = [
    {"model": LinearSVC, "name": "LinearSVC_default", "kwargs": {}},
    {"model": LinearSVC, "name": "LinearSVC_hinge", "kwargs": {"loss": "hinge"}},
    {"model": LinearSVC, "name": "LinearSVC_squared_hinge", "kwargs": {"loss": "squared_hinge"}},
    {"model": LinearSVC, "name": "LinearSVC_C_high", "kwargs": {"C": 10}},
    {"model": LinearSVC, "name": "LinearSVC_C_low", "kwargs": {"C": 0.1}},
    {"model": LinearSVC, "name": "LinearSVC_tol_high", "kwargs": {"tol": 1e-1}},
    {"model": LinearSVC, "name": "LinearSVC_tol_low", "kwargs": {"tol": 1e-5}},
    {"model": LinearSVC, "name": "LinearSVC_penalty_l2", "kwargs": {"penalty": "l2"}},
    {"model": LinearSVC, "name": "LinearSVC_penalty_l1", "kwargs": {"penalty": "l1", "dual": False}},
    {"model": LogisticRegression, "name": "LogisticRegression_default", "kwargs": {}},
    {"model": LogisticRegression, "name": "LogisticRegression_C_high", "kwargs": {"C": 10}},
    {"model": LogisticRegression, "name": "LogisticRegression_C_low", "kwargs": {"C": 0.1}},
    {"model": LogisticRegression, "name": "LogisticRegression_tol_high", "kwargs": {"tol": 1e-1}},
    {"model": LogisticRegression, "name": "LogisticRegression_tol_low", "kwargs": {"tol": 1e-5}},
    {"model": LogisticRegression, "name": "LogisticRegression_penalty_l2", "kwargs": {"penalty": "l2"}},
    {"model": LogisticRegression, "name": "LogisticRegression_penalty_l1", "kwargs": {"penalty": "l1", "solver": "liblinear"}}
]

results = []

for experiment in experiments:
    training_time, accuracy0, accuracy1 = train_and_evaluate(
        X_train, y0_train, y1_train, X_test, y0_test, y1_test, experiment["model"], **experiment["kwargs"]
    )
    results.append({
        "Experiment": experiment["name"],
        "Training Time (s)": training_time,
        "Accuracy0": accuracy0,
        "Accuracy1": accuracy1
    })

import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)


NameError: name 'khatri_rao' is not defined

**Question 6: Code 2**

In [ ]:
  import numpy as np
  import time
  from sklearn.svm import LinearSVC
  from sklearn.linear_model import LogisticRegression
  from sklearn.metrics import accuracy_score
  from sklearn.preprocessing import StandardScaler
  import pandas as pd
  from sklearn.exceptions import ConvergenceWarning
  import warnings
  from scipy.linalg import khatri_rao

  # Load data function
  def load_data(file_path):
      data = np.loadtxt(file_path, delimiter=" ")
      X = data[:, :32]
      y0 = data[:, 32]
      y1 = data[:, 33]
      return X, y0, y1

  # Map challenges to higher dimensional space using Khatri-Rao product

def my_map(X):

    num_features = X.shape[1]
    num_samples = X.shape[0]
    num_mapped_features = num_features ** 2


    mapped_features = np.zeros((num_samples, num_mapped_features))

    for i in range(num_samples):
        row = X[i, :].reshape(1, -1)
        row_result = khatri_rao(row.T, row.T)
        mapped_features[i, :] = row_result.flatten()

    return mapped_features

  # Training and evaluation function
  def train_and_evaluate(X_train, y0_train, y1_train, X_test, y0_test, y1_test, model, **kwargs):
      scaler = StandardScaler()
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      X_train_mapped = my_map(X_train)
      X_test_mapped = my_map(X_test)

      clf0 = model(max_iter=5000, **kwargs)
      clf1 = model(max_iter=5000, **kwargs)

      start_time = time.time()
      with warnings.catch_warnings():
          warnings.simplefilter("ignore", category=ConvergenceWarning)
          clf0.fit(X_train_mapped, y0_train)
          clf1.fit(X_train_mapped, y1_train)
      training_time = time.time() - start_time

      y0_pred = clf0.predict(X_test_mapped)
      y1_pred = clf1.predict(X_test_mapped)

      accuracy0 = accuracy_score(y0_test, y0_pred)
      accuracy1 = accuracy_score(y1_test, y1_pred)

      return training_time, accuracy0, accuracy1

  # Load data
  X_train, y0_train, y1_train = load_data("public_trn.txt")
  X_test, y0_test, y1_test = load_data("public_tst.txt")

  # Experiments setup
  experiments = [
      {"model": LinearSVC, "name": "LinearSVC_default", "kwargs": {}},
      {"model": LinearSVC, "name": "LinearSVC_hinge", "kwargs": {"loss": "hinge"}},
      {"model": LinearSVC, "name": "LinearSVC_squared_hinge", "kwargs": {"loss": "squared_hinge"}},
      {"model": LinearSVC, "name": "LinearSVC_C_high", "kwargs": {"C": 10}},
      {"model": LinearSVC, "name": "LinearSVC_C_low", "kwargs": {"C": 0.1}},
      {"model": LinearSVC, "name": "LinearSVC_tol_high", "kwargs": {"tol": 1e-1}},
      {"model": LinearSVC, "name": "LinearSVC_tol_low", "kwargs": {"tol": 1e-5}},
      {"model": LinearSVC, "name": "LinearSVC_penalty_l2", "kwargs": {"penalty": "l2"}},
      {"model": LinearSVC, "name": "LinearSVC_penalty_l1", "kwargs": {"penalty": "l1", "dual": False}},
      {"model": LogisticRegression, "name": "LogisticRegression_default", "kwargs": {}},
      {"model": LogisticRegression, "name": "LogisticRegression_C_high", "kwargs": {"C": 10}},
      {"model": LogisticRegression, "name": "LogisticRegression_C_low", "kwargs": {"C": 0.1}},
      {"model": LogisticRegression, "name": "LogisticRegression_tol_high", "kwargs": {"tol": 1e-1}},
      {"model": LogisticRegression, "name": "LogisticRegression_tol_low", "kwargs": {"tol": 1e-5}},
      {"model": LogisticRegression, "name": "LogisticRegression_penalty_l2", "kwargs": {"penalty": "l2"}},
      {"model": LogisticRegression, "name": "LogisticRegression_penalty_l1", "kwargs": {"penalty": "l1", "solver": "liblinear"}}
  ]

  results = []

  for experiment in experiments:
      training_time, accuracy0, accuracy1 = train_and_evaluate(
          X_train, y0_train, y1_train, X_test, y0_test, y1_test, experiment["model"], **experiment["kwargs"]
      )
      results.append({
          "Experiment": experiment["name"],
          "Training Time (s)": training_time,
          "Accuracy0": accuracy0,
          "Accuracy1": accuracy1
      })

  results_df = pd.DataFrame(results)
  print(results_df)

Another approach


In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from scipy.linalg import khatri_rao
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

################################
# Non Editable Region Starting #
################################
def my_fit(X_train, y0_train, y1_train):
    ################################
    #  Non Editable Region Ending  #
    ################################

    X_mapped = my_map(X_train)


    clf0 = LogisticRegression(C = 1, tol=0.001 ,dual=False,penalty="l1",solver='liblinear')
    clf0.fit(X_mapped, y0_train)


    clf1 =LogisticRegression(C = 1, tol=0.001 ,dual=False,penalty="l1",solver='liblinear')
    clf1.fit(X_mapped, y1_train)

    w0 = clf0.coef_.flatten()
    b0 = clf0.intercept_[0]

    w1 = clf1.coef_.flatten()
    b1 = clf1.intercept_[0]

    return w0, b0, w1, b1

################################
# Non Editable Region Starting #
################################
def my_map(X):
    ################################
    #  Non Editable Region Ending  #
    ################################
    num_features = X.shape[1]
    num_samples = X.shape[0]
    num_mapped_features = num_features ** 2

    mapped_features = np.zeros((num_samples, num_mapped_features))

    for i in range(num_samples):
        row = X[i, :].reshape(1, -1)
        row_result = khatri_rao(row.T, row.T)
        mapped_features[i, :] = row_result.flatten()

    return mapped_features



Weights and bias for Response0:
[  1.82931984   0.06063988  -0.14471977 ...  -0.87673722   6.96167477
 -12.42754943] -0.7078845712694526
Weights and bias for Response1:
[-0.93684214  0.7490046  -0.37403695 ...  0.2975477   1.07243138
 -1.84041709] -4.640349885983377
Test accuracy for Response0: 0.9827
Test accuracy for Response1: 0.9928
Training time: 17.562750339508057
